In [20]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable


In [21]:
check_gpu = torch.cuda.is_available()
check_gpu

False

In [22]:
# check_devices = torch.cuda.get_device_name(0)
# check_devices

In [23]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [24]:
price_dataset = ohlc_df.close[-1000:]
time_index = ohlc_df.index[-1000:]
print(time_index[:5])
price_dataset.head()


DatetimeIndex(['2022-07-14 07:18:00', '2022-07-14 07:19:00',
               '2022-07-14 07:21:00', '2022-07-14 07:23:00',
               '2022-07-14 07:24:00'],
              dtype='datetime64[ns]', name='time', freq=None)


time
2022-07-14 07:18:00    4.4822
2022-07-14 07:19:00    4.4764
2022-07-14 07:21:00    4.4792
2022-07-14 07:23:00    4.4885
2022-07-14 07:24:00    4.4817
Name: close, dtype: float64

In [25]:
price_dataset_x = price_dataset[200:]
time_x = time_index[200:]
price_dataset_y = price_dataset[200:400]
time_y = time_index[200:400]
print(len(price_dataset_y))

200


In [26]:
price_test_x = price_dataset[400:600]
time_test_x = time_index[400:600]
price_test_y = price_dataset[600:900]
time_test_y = time_index[600:900]

In [96]:
seq_len = 5

In [97]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=5):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        #print(len(self.y))
        X = self.X[index:index+self.seq_len]
        try:
            y = self.y[index+self.seq_len - 1]
        except:
            y = self.y[self.seq_len - 1]
        return X, y

In [98]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=seq_len)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=seq_len)

In [99]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=seq_len, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=seq_len, 
    shuffle=False)

In [113]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, batch_size, output_size, hidden_size, seq_len):
        super(LSTM_Model, self).__init__()
        self.batch_size = batch_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        self.lstm = torch.nn.LSTM(input_size = 1, hidden_size = hidden_size, num_layers = 1)
        self.label = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x, batch_size=None):
        if batch_size is None:
            h_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size)) 
            c_0 = Variable(torch.zeros(1, self.batch_size, self.hidden_size)) 
        else:
            h_0 = Variable(torch.zeros(1, batch_size, self.hidden_size))
            c_0 = Variable(torch.zeros(1, batch_size, self.hidden_size))
        output, (final_hidden_state, final_cell_state) = self.lstm(x, (h_0, c_0))
        final_output = self.label(final_hidden_state[-1]) 
        return final_output


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

crypto_price_model = LSTM_Model(batch_size=1, output_size=1, hidden_size=124, seq_len=seq_len)

if torch.cuda.is_available():
    crypto_price_model.cuda()
crypto_price_model.to(device)

loss_function = torch.nn.CrossEntropyLoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.017) # used adaptive moment estimation to optimize the model

cpu


In [114]:
#training loop
epochs = 5
for i in range(epochs):
    for j,data in enumerate(train_loader):
        #print(data.shape)
       # print(len(data[:]))

        y_pred = crypto_price_model(data[:][0].view(-1,seq_len,1)).reshape(-1)
        #print(y_pred.detach().cpu().numpy())
        
        loss = loss_function(y_pred,data[:][1])
        loss.backward()
        optimizer.step()
    print("Epoch: ", i+1, " Loss: ", loss.item())

RuntimeError: input.size(-1) must be equal to input_size. Expected 5, got 1

In [ ]:
# training_loop(2, optimizer = optimizer, model = crypto_price_model, loss_fn = loss_function, X_train = train_loader, X_val = test_loader, y_train = train_loader, y_val = test_loader)
crypto_price_model.eval()

LSTM_Model(
  (lstm): LSTM(1, 124, batch_first=True)
  (label): Linear(in_features=124, out_features=1, bias=True)
)

In [ ]:
# def clip_gradient(model, clip_value):
#     params = list(filter(lambda p: p.grad is not None, model.parameters()))
#     for p in params:
#         p.grad.data.clamp_(-clip_value, clip_value)
    
# def train_model(model, train_iter, epoch):
#     total_epoch_loss = 0
#     total_epoch_acc = 0
#     model.cuda()
#     optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
#     steps = 0
#     model.train()
#     for idx, batch in enumerate(train_iter):
#         text = batch.text[0]
#         target = batch.label
#         target = torch.autograd.Variable(target).long()
#         if torch.cuda.is_available():
#             text = text.cuda()
#             target = target.cuda()
#         if (text.size()[0] is not 32): 
#             continue
#         optim.zero_grad()
#         prediction = model(text)
#         loss = loss_fn(prediction, target)
#         wandb.log({"Training Loss": loss.item()})
#         num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
#         acc = 100.0 * num_corrects/len(batch)
#         wandb.log({"Training Accuracy": acc.item()})
#         loss.backward()
#         clip_gradient(model, 1e-1)
#         optim.step()
#         steps += 1
        
#         if steps % 100 == 0:
#             print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
#         total_epoch_loss += loss.item()
#         total_epoch_acc += acc.item()
        
#     return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

# def eval_model(model, val_iter):
#     total_epoch_loss = 0
#     total_epoch_acc = 0
#     model.eval()
#     with torch.no_grad():
#         for idx, batch in enumerate(val_iter):
#             text = batch.text[0]
#             if (text.size()[0] is not 32):
#                 continue
#             target = batch.label
#             target = torch.autograd.Variable(target).long()
#             if torch.cuda.is_available():
#                 text = text.cuda()
#                 target = target.cuda()
#             prediction = model(text)
#             loss = loss_fn(prediction, target)
#             wandb.log({"Evaluation Loss": loss.item()})
#             num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
#             acc = 100.0 * num_corrects/len(batch)
#             wandb.log({"Evaluation Accuracy": acc.item()})
#             total_epoch_loss += loss.item()
#             total_epoch_acc += acc.item()

#     return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)

In [ ]:
test_predictions = []
for j,data in enumerate(test_loader):
    #print(data.shape)
    # print(len(data[:]))
    #print(data[0][1])
    y_pred = crypto_price_model(data[:][0].view(-1,seq_len,1)).reshape(-1)
    loss = loss_function(y_pred,data[:][1])
    y_pred = y_pred.detach().cpu().numpy()
    test_predictions.append(y_pred)

In [ ]:
print(len(test_predictions[0]))
print()
fig = go.Figure()
pred_line = go.Scatter(x=time_test_y[:len(test_predictions[0])], y=test_predictions[0], mode = 'lines', name = 'Predicted Price')
fig.add_trace(pred_line)

fig.add_trace(go.Scatter(x=time_test_y[:len(test_predictions[0])], y=price_test_y[:len(test_predictions[0])], mode = 'lines', name = 'Actual Price'))

fig.update_layout(xaxis_title="Time", yaxis_title="Actual Price ($)", title=f"Predicted Price vs Actual Price for '{crypto_name}'")

fig.show()

1



In [ ]:
torch.save(crypto_price_model.state_dict(), f'{crypto_name}_model_weights.pth')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
crypto_price_model.load_state_dict(torch.load(f'{crypto_name}_model_weights.pth', map_location=torch.device(device)))

<All keys matched successfully>